<a href="https://colab.research.google.com/github/vitaldb/respfailure/blob/main/respfailure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Model for Postoperative Respiratory Failure

In [1]:
# recent version xgboost required
!pip uninstall xgboost -y
!pip install xgboost

!wget https://github.com/vitaldb/respfailure/raw/main/model.json

Uninstalling xgboost-0.90:
  Successfully uninstalled xgboost-0.90
     |████████████████████████████████| 166.7MB 76kB/s 
--2021-05-25 14:48:04--  https://github.com/vitaldb/respfailure/raw/main/model.json
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/respfailure/main/model.json [following]
--2021-05-25 14:48:04--  https://raw.githubusercontent.com/vitaldb/respfailure/main/model.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126235 (123K) [text/plain]
Saving to: ‘model.json’

model.json          100%[===================>] 123.28K  --.-KB/s    in 0.01s   

2021-05-25 14:48:04

In [2]:
import xgboost as xgb
import numpy as np
import sys
import pickle

# sample values
age = 56
andur = 162
wbc = 6.2
ptinr = 0.99
na = 141
alb = 4.3

model = xgb.Booster()
model.load_model('model.json')
x = np.array([age, andur, wbc, ptinr, na, alb]).astype(float)
x = x[None, ...]
x = xgb.DMatrix(x)
y = model.predict(x)[0]

print('Risk={:.2f}%'.format(y * 100))

Risk=0.67%
